In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_chroma import Chroma

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import pandas as pd
df = pd.read_csv("data/loan_data_cleaned.csv")
df.head()

,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status,Approval,id,tagged_text
0,I need a loan to pay for an international vaca...,26556,581,8314,79.26,employed,Rejected,0,0 I need a loan to pay for an international va...
1,I want to make home improvements like installi...,197392,389,111604,22.14,employed,Rejected,1,1 I want to make home improvements like instal...
2,"I need a loan for home renovation, including a...",44561,523,34118,45.44,employed,Rejected,2,"2 I need a loan for home renovation, including..."
3,I need funds to buy new furniture and applianc...,190363,729,118757,10.22,unemployed,Rejected,3,3 I need funds to buy new furniture and applia...
4,I need a loan to start a small business.,61853,732,19210,44.13,employed,Approved,4,4 I need a loan to start a small business.


In [12]:
# 显示tagged_text前5行数据
df['tagged_text'].head(7)

0    0 I need a loan to pay for an international va...
1    1 I want to make home improvements like instal...
2    2 I need a loan for home renovation, including...
3    3 I need funds to buy new furniture and applia...
4           4 I need a loan to start a small business.
5    5 I need a loan to repair my car after an acci...
6    6 I need financial help to cover maternity and...
Name: tagged_text, dtype: object

#### 用户通过输入 text来搜索与自己相同的需求，观察需要的其他特征（如：收入，是否就业等情况），从而判断是否可以贷款，当然我也会根据客户的信息进行一个建议性预测。
